# Extract time series parameters from flow and use to predict extreme snowmelt

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import ast
import dateutil.parser as parser

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV, KFold, cross_validate, PredefinedSplit
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import make_scorer, precision_recall_curve, auc, classification_report
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from matplotlib import pyplot as plt


In [ ]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.settings import MinimalFCParameters
from tsfresh.utilities.dataframe_functions import roll_time_series


In [ ]:
# build scorer function
def auc_pr_score(y_true, y_pred):
	precision, recall, _ = precision_recall_curve(y_true, y_pred)
	return auc(recall, precision)


auc_pr = make_scorer(auc_pr_score, greater_is_better=True)

N_DAYS = 5
TIME_LAG = 10

In [ ]:
## Define functions

def random_forest_site(X_under, y_under, results, options):
	indx = X_under.index.unique()
	sites = [x[0] for x in indx]
	sites = list(set(sites))
	sites_train = sites[:len(sites)//10*6]
	sites_test = sites[len(sites)//10*6:]
	idx_train = [x for x in indx if x[0] in sites_train]
	idx_test = [x for x in indx if x[0] in sites_test]
	X_filtered_train = X_under[X_under.index.isin(idx_train)]
	X_filtered_test = X_under[X_under.index.isin(idx_test)]
	y_train = y_under[y_under.index.isin(idx_train)]
	y_test = y_under[y_under.index.isin(idx_test)]

	X_all = pd.concat([X_filtered_train, X_filtered_test]).reset_index(drop=True)
	y_all = pd.concat([y_train, y_test]).reset_index(drop=True)
	clf = RandomForestClassifier(n_jobs=-1, random_state=42, verbose=0)

	split_index = [-1 if x in X_filtered_train.index else 0 for x in X_under.index]
	ps = PredefinedSplit(test_fold=split_index)

	param_grid = {
		'max_depth': (1, 5, 10, 25),
		'n_estimators': (100, 500, 750, 1500),
		'max_features': (2, 3, 5, 10)}

	gs = GridSearchCV(clf, param_grid=param_grid, cv=ps,
					scoring=auc_pr, n_jobs=-1, verbose=0)
	gs.fit(X_all, y_all)
	print(gs.best_params_, gs.best_score_)
	results = results.append({'n_days': N_DAYS, 'time_lag': TIME_LAG, 'eval_type': 'site', 'param_type': options['param_type'], 'year': options['year'],
                           'params': gs.best_params_, 'score': gs.best_score_, 'model': 'Random Forest', 'variables': X_under.columns}, ignore_index=True)
	
	return results

def random_forest_time(X_under, y_under, results, options):
	X_filtered_sorted = X_under.sort_index(key=lambda d: d.map(lambda x: x[1]))
	y_under_sorted = y_under.sort_index(key=lambda d: d.map(lambda x: x[1]))
	tscv = TimeSeriesSplit(n_splits=5)

	param_grid = {
		'max_depth': (1, 5, 10, 25),
		'n_estimators': (100, 500, 750, 1500),
		'max_features': (2, 3, 5, 10)}

	clf = RandomForestClassifier(n_jobs=-1, random_state=42, verbose=0)

	gs = GridSearchCV(clf, param_grid=param_grid, cv=tscv,
					scoring=auc_pr, n_jobs=-1, verbose=0)
	gs.fit(X_filtered_sorted, y_under_sorted)
	
	results = results.append({'n_days': N_DAYS, 'time_lag': TIME_LAG, 'eval_type': 'time', 'param_type': options['param_type'], 'year': options['year'], 
                           'params': gs.best_params_, 'score': gs.best_score_, 'model': 'Random Forest', 'variables': X_under.columns}, ignore_index=True)
	print(gs.best_params_, gs.best_score_)

	return results


def gradient_boost_site(X_under, y_under, results, options):
	indx = X_under.index.unique()
	sites = [x[0] for x in indx]
	sites = list(set(sites))
	sites_train = sites[:len(sites)//10*6]
	sites_test = sites[len(sites)//10*6:]
	idx_train = [x for x in indx if x[0] in sites_train]
	idx_test = [x for x in indx if x[0] in sites_test]
	X_filtered_train = X_under[X_under.index.isin(idx_train)]
	X_filtered_test = X_under[X_under.index.isin(idx_test)]
	y_train = y_under[y_under.index.isin(idx_train)]
	y_test = y_under[y_under.index.isin(idx_test)]

	X_all = pd.concat([X_filtered_train, X_filtered_test]).reset_index(drop=True)
	y_all = pd.concat([y_train, y_test]).reset_index(drop=True)
	clf = HistGradientBoostingClassifier(random_state=42, verbose=0, early_stopping=False)

	split_index = [-1 if x in X_filtered_train.index else 0 for x in X_under.index]
	ps = PredefinedSplit(test_fold=split_index)

	param_grid = {'max_iter': (100, 1000, 1500),
               'learning_rate': (0.01, 0.1, 1),
               'max_depth': (1, 5, 10, 25, 50),
               }

	gs = GridSearchCV(clf, param_grid=param_grid, cv=ps,
                   scoring=auc_pr, n_jobs=-1, verbose=0)
	gs.fit(X_all, y_all)
	print(gs.best_params_, gs.best_score_)
	results = results.append({'n_days': N_DAYS, 'time_lag': TIME_LAG, 'eval_type': 'site', 'param_type': options['param_type'], 'year': options['year'],
                           'params': gs.best_params_, 'score': gs.best_score_, 'model': 'Gradient Boost', 'variables': X_under.columns}, ignore_index=True)

	return results


def gradient_boost_time(X_under, y_under, results, options):
	X_filtered_sorted = X_under.sort_index(key=lambda d: d.map(lambda x: x[1]))
	y_under_sorted = y_under.sort_index(key=lambda d: d.map(lambda x: x[1]))
	tscv = TimeSeriesSplit(n_splits=5)

	param_grid = {'max_iter': (100, 1000, 1500),
               'learning_rate': (0.01, 0.1, 1),
               'max_depth': (1, 5, 10, 25, 50),
               }
	clf = HistGradientBoostingClassifier(random_state=42, verbose=0, early_stopping=False)

	gs = GridSearchCV(clf, param_grid=param_grid, cv=tscv,
                   scoring=auc_pr, n_jobs=-1, verbose=0)
	gs.fit(X_filtered_sorted, y_under_sorted)

	results = results.append({'n_days': N_DAYS, 'time_lag': TIME_LAG, 'eval_type': 'time', 'param_type': options['param_type'], 'year': options['year'],
                           'params': gs.best_params_, 'score': gs.best_score_, 'model': 'Gradient Boost', 'variables': X_under.columns}, ignore_index=True)
	print(gs.best_params_, gs.best_score_)

	return results


## Create rolled dataframe 

In [ ]:
all_data_clean = pd.read_csv('../all_data_clean.csv')

all_data_clean


In [ ]:

## Here can change parameters only once
df_rolled = roll_time_series(
    all_data_clean[['date', 'flow_site_id', 'flow', 'temp', 'prec', 'binary']], column_id="flow_site_id", column_sort="date", max_timeshift=N_DAYS, min_timeshift=N_DAYS - 1, n_jobs=20)


In [ ]:
df_rolled

In [ ]:
df_rolled.to_csv('../df_rolled_' + str(N_DAYS) + '.csv', index=False)

## Extract minimal timeseries features

In [ ]:
df_rolled = pd.read_csv('../df_rolled_' + str(N_DAYS) + '.csv')
all_data_clean = pd.read_csv('../all_data_clean.csv')

In [ ]:
df_rolled

In [ ]:
# extract timeseries features

X_features_all = extract_features(
	df_rolled.drop(["binary", "flow_site_id"], axis=1), column_id='id', column_sort='date',
	n_jobs=20, disable_progressbar=False, default_fc_parameters=MinimalFCParameters())


X_features_all.head()


In [ ]:
## Add binary response variable back based on unique id

X_features_all['unique_id'] = X_features_all.index
X_features_all['unique_id'] = X_features_all['unique_id'].apply(ast.literal_eval)

all_data_clean['shifted_date'] = pd.to_datetime(
    all_data_clean.date) + pd.Timedelta(days=TIME_LAG)
all_data_clean['shifted_date'] = all_data_clean['shifted_date'].dt.strftime(
    '%Y-%m-%d')
all_data_clean['unique_id'] = list(
    zip(all_data_clean.flow_site_id, all_data_clean.shifted_date))
all_data_clean = all_data_clean.dropna()

X_features_all = X_features_all.reset_index(drop=True)
	
X_features_all = pd.merge(X_features_all, all_data_clean[[
                          'binary', 'unique_id']], how='left', on='unique_id')
X_features_all = X_features_all.set_index(
    X_features_all['unique_id'], drop=True)
X_features_all = X_features_all.dropna()
X_features_all.head()


In [ ]:
X_features_all.to_csv('../df_extracted_min_' + str(N_DAYS) + '.csv', index=False)

## Undersample minimal timeseries feature dataset and run Random Forest model

In [ ]:
X_features_all = pd.read_csv('../df_extracted_min_' + str(N_DAYS) + '.csv')


In [ ]:
X_features_all['unique_id'] = X_features_all['unique_id'].apply(ast.literal_eval)
X_features_all.head()


In [ ]:
y1 = X_features_all['binary']
Counter(y1)

In [ ]:
## undersample

from imblearn.under_sampling import NearMiss

undersample = NearMiss(version=3, n_neighbors=3)
X_under, y_under = undersample.fit_resample(
    X_features_all.drop(columns=['binary', 'unique_id']), y1)


In [ ]:
X_under.index = X_features_all['unique_id'][undersample.sample_indices_]
y_under.index = X_features_all['unique_id'][undersample.sample_indices_]
Counter(y_under)


In [ ]:
results = pd.DataFrame(columns=['n_days', 'time_lag', 'eval_type', 'param_type', 'year', 'params', 'score', 'model', 'variables'])

In [ ]:
# Select relevant features for random forest
sel = SelectFromModel(RandomForestClassifier(n_jobs=-1, random_state=42))
sel.fit(X_under, y_under)
selected_feat = X_under.columns[(sel.get_support())]
X_selected = X_under[selected_feat]


In [ ]:
# Remove correlated features
corr_matrix = X_selected.corr().abs()
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
X_selected.drop(to_drop, axis=1, inplace=True)
X_selected


In [ ]:
## run model without year and split by site and time
results = random_forest_site(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'No'})
results = random_forest_time(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'No'})


In [ ]:
## run model without year and split by site and time
results = gradient_boost_site(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'No'})
results = gradient_boost_time(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'No'})


In [ ]:
## add year to features
dates = [parser.parse(x[1]).year for x in X_selected.index]
X_selected['year'] = dates


In [ ]:
## run model with year and split by site and time
results = random_forest_site(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'Yes'})
results = random_forest_time(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'Yes'})


In [ ]:
## run model with year and split by site and time
results = gradient_boost_site(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'Yes'})
results = gradient_boost_time(X_selected, y_under, results, options={'param_type': 'Minimal', 'year': 'Yes'})


In [ ]:
df_rolled['id'] = df_rolled['id'].apply(ast.literal_eval)
df_rolled.id

In [ ]:
X_under_all = df_rolled[df_rolled.id.isin(X_under.index)]
X_under_all


In [ ]:
X_under_all.to_csv('../df_undersampled_nearmiss_' +
                   str(N_DAYS) + '_time_lag_' + str(TIME_LAG) + '.csv', index=False)


## Extract complete set of timeseries features

In [ ]:
X_under_all = pd.read_csv('../df_undersampled_nearmiss_' +
                          str(N_DAYS) + '_time_lag_' + str(TIME_LAG) + '.csv')
X_under_all.head()

In [ ]:
# extract timeseries features

X_features_all = extract_features(
	X_under_all.drop(["binary", "flow_site_id"], axis=1), column_id='id', column_sort='date',
	n_jobs=20, disable_progressbar=False)


X_features_all.head()

In [ ]:
X_features_all = X_features_all.dropna(axis=1)
X_features_all['unique_id'] = X_features_all.index

In [ ]:
X_features_all.to_csv(
    '../df_extracted_all_nearmiss_' + str(N_DAYS) + '_time_lag_' + str(TIME_LAG) + '.csv', index=False)


## Build Random Forest model with complete set of extracted timeseries parameters

In [ ]:
X_features_all = pd.read_csv(
    '../df_extracted_all_nearmiss_' + str(N_DAYS) + '_time_lag_' + str(TIME_LAG) + '.csv')

all_data_clean = pd.read_csv('../all_data_clean.csv')

### to get the binary labels for n days after end of rolled time series
all_data_clean['shifted_date'] = pd.to_datetime(
    all_data_clean.date) + pd.Timedelta(days=TIME_LAG)
all_data_clean['shifted_date'] = all_data_clean['shifted_date'].dt.strftime(
    '%Y-%m-%d')
all_data_clean['unique_id'] = list(
    zip(all_data_clean.flow_site_id, all_data_clean.shifted_date))
all_data_clean = all_data_clean.dropna()
all_data_clean.head()


In [ ]:
X_features_all['unique_id'] = X_features_all['unique_id'].apply(ast.literal_eval)


In [ ]:
X_features_under_all = pd.merge(X_features_all, all_data_clean[[
    'binary', 'year', 'unique_id']], how='left', on='unique_id')
X_features_under_all = X_features_under_all.set_index(
    X_features_under_all['unique_id'], drop=True)
y_under = X_features_under_all['binary']
X_features_under_filtered  =X_features_under_all.replace(np.inf, np.nan)
X_features_under_filtered = X_features_under_filtered.dropna(axis=1)
X_features_under_filtered = X_features_under_filtered.drop(
    columns=['unique_id', 'binary'])
X_features_under_filtered.head()


In [ ]:
## run model without year and split by site and time
#results = random_forest_site(X_features_under_filtered.drop(columns=['year']), y_under, results, options={'param_type': 'All', 'year': 'No'})
#results = random_forest_time(X_features_under_filtered.drop(columns=['year']), y_under, results, options={'param_type': 'All', 'year': 'No'})


In [ ]:
## run model without year and split by site and time
#results = gradient_boost_site(X_features_under_filtered.drop(columns=['year']), y_under, results, options={'param_type': 'All', 'year': 'No'})
#results = gradient_boost_time(X_features_under_filtered.drop(columns=['year']), y_under, results, options={'param_type': 'All', 'year': 'No'})

In [ ]:
## run model with year and split by site and time
#results = random_forest_site(X_features_under_filtered, y_under, results, options={'param_type': 'All', 'year': 'Yes'})
#results = random_forest_time(X_features_under_filtered, y_under, results, options={'param_type': 'All', 'year': 'Yes'})


In [ ]:
## run model with year and split by site and time
#results = gradient_boost_site(X_features_under_filtered, y_under, results, options={'param_type': 'All', 'year': 'Yes'})
#results = gradient_boost_time(X_features_under_filtered, y_under, results, options={'param_type': 'All', 'year': 'Yes'})

### Run the model with selected features

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_jobs=-1, random_state=42))
sel.fit(X_features_under_filtered.drop(columns=['year']), y_under)
selected_feat = X_features_under_filtered.drop(
    columns=['year']).columns[(sel.get_support())]
X_selected = X_features_under_filtered[selected_feat]
X_selected['year'] = X_features_under_filtered['year']

In [ ]:
# Remove correlated features
corr_matrix = X_selected.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
X_selected.drop(to_drop, axis=1, inplace=True)
X_selected

In [ ]:
## run model without year and split by site and time
results = random_forest_site(X_selected.drop(columns=['year']), y_under, results, options={'param_type': 'Selected', 'year': 'No'})
results = random_forest_time(X_selected.drop(columns=['year']), y_under, results, options={'param_type': 'Selected', 'year': 'No'})

In [ ]:
## run model without year and split by site and time
results = gradient_boost_site(X_selected.drop(columns=['year']), y_under, results, options={'param_type': 'Selected', 'year': 'No'})
results = gradient_boost_time(X_selected.drop(columns=['year']), y_under, results, options={'param_type': 'Selected', 'year': 'No'})


In [ ]:
## run model with year and split by site and time
results = random_forest_site(X_selected, y_under, results, options={'param_type': 'Selected', 'year': 'Yes'})
results = random_forest_time(X_selected, y_under, results, options={'param_type': 'Selected', 'year': 'Yes'})

In [ ]:
## run model with year and split by site and time
results = gradient_boost_site(X_selected, y_under, results, options={'param_type': 'Selected', 'year': 'Yes'})
results = gradient_boost_time(X_selected, y_under, results, options={'param_type': 'Selected', 'year': 'Yes'})


In [ ]:
results.to_csv('../results_' + str(N_DAYS) + '_time_lag_' + str(TIME_LAG) + '_flow_temp_prec.csv')